## Training

In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
# !git clone https://github.com/huggingface/transformers.git
# import os
# os.chdir('transformers')
# !pip install -e .

In [3]:
!pip install git+https://github.com/huggingface/accelerate

  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-lrd9fxlj
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-lrd9fxlj
  Resolved https://github.com/huggingface/accelerate to commit 127818fc27ebe5cb236357fff59ff1748326d643
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [4]:
!pip install datasets evaluate torch>=1.3 transformers accelerate>=0.12.0 sentencepiece!=0.1.92 protobuf tqdm

In [5]:
!pip install kaggle

In [6]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {"username":"ruby0322","key":"949a390a18b790a080fde3812391d784"}

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [7]:
!kaggle competitions download -c ntu-adl-2024-hw-1-chinese-extractive-qa

  0% 0.00/7.64M [00:00<?, ?B/s]
100% 7.64M/7.64M [00:00<00:00, 97.6MB/s]


In [8]:
!unzip ntu-adl-2024-hw-1-chinese-extractive-qa.zip

Archive:  ntu-adl-2024-hw-1-chinese-extractive-qa.zip
  inflating: context.json            
  inflating: sample_submission.csv   
  inflating: test.json               
  inflating: train.json              
  inflating: valid.json              


In [9]:
train = pd.read_json('train.json')
train = train.drop(columns=['id'])
valid = pd.read_json('valid.json')
valid = valid.drop(columns=['id'])

In [10]:
contexts_df = pd.read_json('context.json')
contexts = np.array(contexts_df[0])

In [11]:
def transform_to_training_format(df, contexts):
    formatted_data = []

    # Loop over each row in the DataFrame
    for index, row in df.iterrows():
        question = row['question']
        paragraphs = row['paragraphs']  # These are indices of contexts
        relevant = row['relevant']

        # Create the format
        entry = {
            "sent1": question,
            "ending0": contexts[paragraphs[0]],
            "ending1": contexts[paragraphs[1]],
            "ending2": contexts[paragraphs[2]],
            "ending3": contexts[paragraphs[3]],
            "label": paragraphs.index(relevant)  # Find the index of the relevant context
        }

        # Append the formatted entry
        formatted_data.append(entry)

    return formatted_data

In [12]:
train_data = transform_to_training_format(train, contexts)
valid_data = transform_to_training_format(valid, contexts)

In [13]:
with open('train_data.json', 'w') as train_data_json:
    json.dump(train_data, train_data_json)
with open('valid_data.json', 'w') as valid_data_json:
    json.dump(valid_data, valid_data_json)
# valid_data.to_json(orient='records')
# test_data.to_json(orient='records')

In [47]:
!rm -rf ./csie5431-applied-data-learning
!git clone https://github.com/ruby0322/csie5431-applied-data-learning.git

Cloning into 'csie5431-applied-data-learning'...
remote: Enumerating objects: 78, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 78 (delta 32), reused 73 (delta 27), pack-reused 0 (from 0)
Receiving objects: 100% (78/78), 31.06 KiB | 1.24 MiB/s, done.
Resolving deltas: 100% (32/32), done.


In [48]:
!mv ./csie5431-applied-data-learning/hw1/multiple_choice_train.py ./
!mv ./csie5431-applied-data-learning/hw1/multiple_choice_inference.py ./
!mv ./csie5431-applied-data-learning/hw1/multiple_choice_eval.py ./
!mkdir ./output

mkdir: cannot create directory ‘./output’: File exists


In [49]:
!python multiple_choice_train.py \
  --train_file train_data.json \
  --validation_file valid_data.json \
  --model_name_or_path hfl/chinese-roberta-wwm-ext \
  --tokenizer_name hfl/chinese-roberta-wwm-ext \
  --output_dir ./output \
  --per_device_train_batch_size 4 \
  --per_device_eval_batch_size 4 \
  --learning_rate 3e-5 \
  --num_train_epochs 3 \
  --max_seq_length 512 \
  --pad_to_max_length \
  --lr_scheduler_type cosine \
  --seed 42

2024-10-08 13:48:31.949559: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-08 13:48:31.966987: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-08 13:48:31.988906: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-08 13:48:31.995405: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-08 13:48:32.011290: I tensorflow/core/platform/cpu_feature_guar

In [50]:
!zip -r output.zip ./output

  adding: output/ (stored 0%)
  adding: output/config.json (deflated 54%)
  adding: output/model.safetensors (deflated 7%)
  adding: output/tokenizer.json (deflated 75%)
  adding: output/special_tokens_map.json (deflated 80%)
  adding: output/vocab.txt (deflated 48%)
  adding: output/tokenizer_config.json (deflated 75%)
  adding: output/all_results.json (stored 0%)


In [18]:
# from google.colab import files
# files.download('./output.zip')

In [26]:
def transform_to_testing_format(df, contexts):
    formatted_data = []

    # Loop over each row in the DataFrame
    for index, row in df.iterrows():
        question = row['question']
        paragraphs = row['paragraphs']  # These are indices of contexts

        # Create the format
        entry = {
            "id": row['id'],
            "sent1": question,
            "ending0": contexts[paragraphs[0]],
            "ending1": contexts[paragraphs[1]],
            "ending2": contexts[paragraphs[2]],
            "ending3": contexts[paragraphs[3]],
        }

        # Append the formatted entry
        formatted_data.append(entry)

    return formatted_data

In [29]:
test = pd.read_json('test.json')

In [34]:
test_data = transform_to_testing_format(test, contexts)

In [31]:
with open('test_data.json', 'w') as test_data_json:
    json.dump(test_data, test_data_json)

In [22]:
!gdown 1NKYF5YR9RYJFXFehrHBtT2o5CsloEwOg

Downloading...
From (original): https://drive.google.com/uc?id=1NKYF5YR9RYJFXFehrHBtT2o5CsloEwOg
From (redirected): https://drive.google.com/uc?id=1NKYF5YR9RYJFXFehrHBtT2o5CsloEwOg&confirm=t&uuid=a5c07fdd-ddcc-43a4-bbf9-77d8d7b96dca
To: /content/multiple-choice-fine-tune-96.zip
100% 380M/380M [00:08<00:00, 45.3MB/s]


In [23]:
!unzip ./multiple-choice-fine-tune-96.zip

Archive:  ./multiple-choice-fine-tune-96.zip
   creating: multiple-choice-fine-tune-96/
  inflating: __MACOSX/._multiple-choice-fine-tune-96  
  inflating: multiple-choice-fine-tune-96/model.safetensors  
  inflating: __MACOSX/multiple-choice-fine-tune-96/._model.safetensors  
  inflating: multiple-choice-fine-tune-96/tokenizer_config.json  
  inflating: __MACOSX/multiple-choice-fine-tune-96/._tokenizer_config.json  
  inflating: multiple-choice-fine-tune-96/special_tokens_map.json  
  inflating: __MACOSX/multiple-choice-fine-tune-96/._special_tokens_map.json  
  inflating: multiple-choice-fine-tune-96/config.json  
  inflating: __MACOSX/multiple-choice-fine-tune-96/._config.json  
  inflating: multiple-choice-fine-tune-96/tokenizer.json  
  inflating: __MACOSX/multiple-choice-fine-tune-96/._tokenizer.json  
  inflating: multiple-choice-fine-tune-96/vocab.txt  
  inflating: __MACOSX/multiple-choice-fine-tune-96/._vocab.txt  
  inflating: multiple-choice-fine-tune-96/all_results.json  


## Inference

In [38]:
!pip install numpyencoder

In [39]:
!python multiple_choice_inference.py \
    --test_file test_data.json \
    --model_dir ./multiple-choice-fine-tune-96 \
    --output_file predictions.json


2024-10-08 12:59:50.822064: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-08 12:59:50.840090: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-08 12:59:50.861436: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-08 12:59:50.867859: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-08 12:59:50.883365: I tensorflow/core/platform/cpu_feature_guar